# Ours

our code

In [4]:
import os
import math
from shutil import copyfile

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import datasets, transforms
import torch.backends.cudnn as cudnn
from torchvision.transforms import InterpolationMode
from torchvision import models

import numpy as np
import random





In [47]:
from utils.common import setup_seed 
from utils.loader import environments, init_dataset_train, tensor2label, label2tensor, init_dataset_test
from torch import nn
from tqdm import tqdm
from torch.utils.data import Subset, DataLoader


class DomainClassifier(nn.Module):
    def __init__(self, domin_num=10) -> None:
        super(DomainClassifier, self).__init__()
        
        # adatped from modern backbone, change last fc to adopt domain
        self.net = models.resnet18(pretrained=True)
        ftr_num = self.net.fc.in_features
        self.net.fc = nn.Linear(ftr_num, domin_num)

    def forward(self, x):
        return self.net(x)
    
    

class Ours:
    def __init__(self, 
                 use_wandb=True,
                 wandb_key = '16c9a3f92163ef4df08841029e02fded0cd0cfed'
                 ) -> None:
        self.seed = 2024
        self.use_wandb = use_wandb # use wandb to monitor training instead of CLI
        self.wandb_key = wandb_key
        # init
        setup_seed(self.seed)
        self.model_dir = os.path.join(os.getcwd(), 'model', 'Ours')
        os.environ['TORCH_HOME']='./' 
        
        # 
        # self.domain_classifer = DomainClassifier() 

    def train_domain_classifier(self,
                name='DomainClassifier',
                lr=0.001,
                num_epochs=60,
                batchsize=16,
                h=256,
                w=256,
                train_num=None,
                dataset='University-Release',
                checkpoint_interval=20,
                checkpoint_start=10,
                ):
        # load Dataset
        image_datasets, dataloaders, dataset_sizes = init_dataset_train(name=dataset, w=w, h=h, style='mixed')

        if type(train_num) is int:
            indices = list(range(train_num))
            s_dataset = Subset(image_datasets['drone'], indices)
        else:
            s_dataset = image_datasets['drone']
        s_dataloader = DataLoader(s_dataset, batch_size=batchsize, shuffle=True, num_workers=0) 

        # Create Model
        model = DomainClassifier()
        model = model.cuda()
        model.train(True)  # Set model to training mode

        optimizer = optim.Adam(model.parameters(), lr=lr)
        criterion = nn.CrossEntropyLoss()

        # trivial settings
        # use wandb to log
        if self.use_wandb:
            import wandb
            os.environ["WANDB_API_KEY"] = self.wandb_key
            wandb.init(project="ACMMMW24", name=name)
        # save pth
        if not os.path.isdir(self.model_dir):
            os.mkdir(self.model_dir)

        
        for epoch in range(1, num_epochs+1):     
            correct = 0 
            total = 0
            epoch_loss = 0
            for data in tqdm(s_dataloader, ):                                
                # get the inputs
                inputs, labels, weather = data
                now_batch_size,c,h,w = inputs.shape
                if now_batch_size<batchsize: # skip the last batch
                    continue
                wt = label2tensor(weather)
                if torch.cuda.is_available():
                    inputs = inputs.cuda().detach()
                    labels = labels.cuda().detach()
                    wt = wt.cuda().detach()

                # zero the parameter gradients
                optimizer.zero_grad()

                wo = model(inputs)
                loss = criterion(wo, wt)
                sssss = tensor2label(wt)
                xxxxx = tensor2label(wo)

                for i in range(batchsize):
                    if sssss[i] == xxxxx[i]:
                        correct = correct + 1
                total = total + batchsize
                epoch_loss += float(loss.cpu().detach().numpy())
                loss.backward()
                optimizer.step() 
            if self.use_wandb:
                wandb.log({'Loss': epoch_loss, 'Acc': correct/total})
            print("Epoch: {} Loss: {:.4f} Acc: {:.4f}".format(epoch, epoch_loss, correct/total))

            if epoch % checkpoint_interval == 0 and epoch > checkpoint_start:     
                save_filename = '{}_{:03d}.pth'.format(name, epoch)
                save_path = os.path.join(os.path.join(os.getcwd(), "model", name, save_filename))
                torch.save(model.cpu().state_dict(), save_path)
                model.cuda() # essential!

    def test_domain_classifier_single_batch(self,
                                      dataset='University-Release',
                                      h=256,
                                      w=256,
                                      batchsize=8,
                                      model_file='DomainClassifier_060.pth'
                                      ):
        # load Dataset
        image_datasets, dataloaders, dataset_sizes = init_dataset_test(name=dataset, w=w, h=h, style='mixed', batchsize=batchsize)
        dataiter = iter(dataloaders['query_drone'])
        img, label, weather = dataiter.next()
        #print(img.shape)
        # print(label)
        
        domain_classifier = DomainClassifier()
        domain_classifier.load_state_dict(torch.load(os.path.join(self.model_dir, model_file))) 
        logits = domain_classifier(img) 
        pred = torch.softmax(logits, dim=1)

        print(logits[0])
        print(pred[0])


        print("GT:        ", weather)
        print("predicted: ", tensor2label(logits))
            


In [48]:
a = Ours(use_wandb=False)


In [49]:
# a.train_domain_classifier(num_epochs=3,checkpoint_interval=1, checkpoint_start=0)
a.test_domain_classifier_single_batch()

c:\Users\lyq\.conda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\Users\lyq\.conda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


tensor([-16.9710, -52.0030, -30.8086, -35.0373, -18.2553, -50.1074, -20.0993,
        -28.2631,   5.9438, -30.4980], grad_fn=<SelectBackward0>)
tensor([1.1174e-10, 6.8235e-26, 1.0929e-16, 1.5927e-18, 3.0935e-11, 4.5423e-25,
        4.8933e-12, 1.3936e-15, 1.0000e+00, 1.4911e-16],
       grad_fn=<SelectBackward0>)
GT:         ('light', 'normal', 'normal', 'snow', 'rain_snow', 'wind', 'dark', 'rain')
predicted:  ['light', 'normal', 'normal', 'snow', 'rain_snow', 'wind', 'dark', 'rain']


```
conda activate pytorch
wandb sync d:\acmmm24-ours\wandb\run-20240618_124948-u09o94cy
16c9a3f92163ef4df08841029e02fded0cd0cfed
```

In [43]:
import imgaug.augmenters as iaa

import torch.utils.data as Data
from LPN.image_folder import *

environments = {'normal': iaa.Sequential([iaa.Noop()]),
                'dark' : iaa.Sequential([
                                        # iaa.BlendAlpha(0.5, foreground=iaa.Add(100), background=iaa.Multiply(0.2), seed=31),
                                        iaa.MultiplyAndAddToBrightness(mul=0.4, add=-15, seed=1991)]),
                'fog'  : iaa.Sequential([iaa.CloudLayer(intensity_mean=225, intensity_freq_exponent=-2, intensity_coarse_scale=2, alpha_min=1.0,
                                        alpha_multiplier=0.9, alpha_size_px_max=10, alpha_freq_exponent=-2, sparsity=0.9, density_multiplier=0.5, seed=35)]),
                'rain' : iaa.Sequential([iaa.Rain(drop_size=(0.05, 0.1), speed=(0.04, 0.06), seed=38),
                                        iaa.Rain(drop_size=(0.05, 0.1), speed=(0.04, 0.06), seed=35),
                                        iaa.Rain(drop_size=(0.1, 0.2), speed=(0.04, 0.06), seed=73),
                                        iaa.Rain(drop_size=(0.1, 0.2), speed=(0.04, 0.06), seed=93),
                                        iaa.Rain(drop_size=(0.05, 0.2), speed=(0.04, 0.06), seed=95)]),
                'snow' : iaa.Sequential([iaa.Snowflakes(flake_size=(0.5, 0.8), speed=(0.007, 0.03), seed=38),
                                        iaa.Snowflakes(flake_size=(0.5, 0.8), speed=(0.007, 0.03), seed=35),
                                        iaa.Snowflakes(flake_size=(0.6, 0.9), speed=(0.007, 0.03), seed=74),
                                        iaa.Snowflakes(flake_size=(0.6, 0.9), speed=(0.007, 0.03), seed=94),
                                        iaa.Snowflakes(flake_size=(0.5, 0.9), speed=(0.007, 0.03), seed=96)]),
                'fog_rain' : iaa.Sequential([iaa.CloudLayer(intensity_mean=225, intensity_freq_exponent=-2, intensity_coarse_scale=2, alpha_min=1.0,
                                            alpha_multiplier=0.9, alpha_size_px_max=10, alpha_freq_exponent=-2, sparsity=0.9, density_multiplier=0.5, seed=35),
                                            iaa.Rain(drop_size=(0.05, 0.2), speed=(0.04, 0.06), seed=35),
                                            iaa.Rain(drop_size=(0.05, 0.2), speed=(0.04, 0.06), seed=36)]),
                'fog_snow' : iaa.Sequential([iaa.CloudLayer(intensity_mean=225, intensity_freq_exponent=-2, intensity_coarse_scale=2, alpha_min=1.0,
                                            alpha_multiplier=0.9, alpha_size_px_max=10, alpha_freq_exponent=-2, sparsity=0.9, density_multiplier=0.5, seed=35),
                                            iaa.Snowflakes(flake_size=(0.5, 0.9), speed=(0.007, 0.03), seed=35),
                                            iaa.Snowflakes(flake_size=(0.5, 0.9), speed=(0.007, 0.03), seed=36)]),
                'rain_snow' : iaa.Sequential([iaa.Snowflakes(flake_size=(0.5, 0.8), speed=(0.007, 0.03), seed=35),
                                            iaa.Rain(drop_size=(0.05, 0.1), speed=(0.04, 0.06), seed=35),
                                            iaa.Rain(drop_size=(0.1, 0.2), speed=(0.04, 0.06), seed=92),
                                            iaa.Rain(drop_size=(0.05, 0.2), speed=(0.04, 0.06), seed=91),
                                            iaa.Snowflakes(flake_size=(0.6, 0.9), speed=(0.007, 0.03), seed=74)]),
                'light': iaa.Sequential([iaa.MultiplyAndAddToBrightness(mul=1.6, add=(0, 30), seed=1992)]),
                'wind' : iaa.Sequential([iaa.MotionBlur(15, seed=17)])
                }

environments2index = {env: idx for idx, env in enumerate(environments)}
index2environments = [i for i in environments]

class WeatherTransform:
    def __init__(self, aug='normal') -> None:
        self.transform = environments[aug]

    def __call__(self, img):
        img = np.array(img) # input is PIL
        img = self.transform(image=img)
        img = Image.fromarray(img)
        return img
    

class MyDataset(datasets.ImageFolder):
    # enable various weather enhancement
    # use for drone-view only
    def __init__(self, root, transform = None, target_transform = None, style='normal', h=384, w=384):
        super().__init__(root, transform = transform, target_transform = target_transform)
        self.envir_list = [i for i in environments]
        self.style_list = self.envir_list + ['mixed']
        assert style in self.style_list, f"style must be one of {self.style_list}"
        self.style = style 
        # assert stage in ['train', 'test'], f"style must be one of {['train', 'test']}"
        # self.stage = stage
         
        # transform setting
        self.h = h 
        self.w = w

    def __getitem__(self, index):
        img, target = super().__getitem__(index)
        # enhance image according to self.style 
        h = self.h 
        w = self.w 

        if self.style=='mixed':
            weather = np.random.choice(self.envir_list)
        else:
            weather = self.style


        t = transforms.Compose(
            [
                transforms.Resize((h, w), interpolation=InterpolationMode.BICUBIC),
                WeatherTransform(aug=weather),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]
        )
        
        img = t(img)
        return img, target, weather    

    def __len__(self):
        return len(self.imgs)
    
def label2tensor(label, num_classes=10):
    idxs = [environments2index[i] for i in label]
    idxs = torch.tensor(idxs) 
    t = torch.nn.functional.one_hot(idxs, num_classes=num_classes).float()
    return t

def tensor2label(t):
    indices = torch.argmax(t, dim=1)
    return [index2environments[i] for i in indices]



Test custom dataset(skip freely)

In [ ]:
a = MyDataset(os.path.join(os.getcwd(), 'University-Release', 'train', 'drone'), style='mixed')

from matplotlib import pyplot as plt
import numpy as np

def imshow(img):
    img = img * torch.tensor([0.229, 0.224, 0.225]).view(3,1,1) + torch.tensor([0.485, 0.456, 0.406]).view(3,1,1)
    img = img.numpy()
    plt.imshow(np.transpose(img, (1,2,0)))
    plt.show() 

d = torch.utils.data.DataLoader(a, batch_size=8, shuffle=True, num_workers=0, pin_memory=False)
dataiter = iter(d)
img, label, weather = dataiter.next()
#print(img.shape)
print(label)

print(weather)
x = label2tensor(weather)
xx = tensor2label(x)
print(xx)



imshow(img[0])

## build a CNN network to classify weather

In [60]:
from torch import nn
from tqdm import tqdm
from torch.utils.data import Subset, DataLoader

class WeatherCNN(nn.Module):
    def __init__(self, class_num=10, h=384, w=384, latent_dim=1000):
        super(WeatherCNN, self).__init__()
        self.w = w 
        self.h = h

        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )
        fc_input_size = self.get_fc_input_size() # to dynamically calculate fc input size, since h,w changes
        self.fc1 = nn.Linear(fc_input_size, latent_dim)
        self.fc2 = nn.Linear(latent_dim, class_num)
        self.softmax = nn.Softmax(dim=-1)      

    def forward(self, xx):
        x = self.layer1(xx)
        x = self.layer2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x
    
    def get_fc_input_size(self):
        dummy_imput = torch.zeros(1, 3, self.h, self.w)
        x = self.layer1(dummy_imput)
        x = self.layer2(x)
        x = x.view(x.size(0), -1)
        return x.shape[1]
    
def pretrain_WeatherCNN(
                name='WeatherCNN',
                lr=0.001,
                num_epochs=60,
                batchsize=16,
                h=256,
                w=256,
                train_num=None,
                dataset='University-Release', 
                style='mixed'):
    # load Dataset
    image_datasets, dataloaders, dataset_sizes = init_dataset_train(name=dataset, w=w, h=h, style=style)

    if type(train_num) is int:
        indices = list(range(train_num))
        s_dataset = Subset(image_datasets['drone'], indices)
    else:
        s_dataset = image_datasets['drone']
    s_dataloader = DataLoader(s_dataset, batch_size=batchsize, shuffle=True, num_workers=0) 

    # Create Model
    model = WeatherCNN(h=h, w=w)
    model = model.cuda()
    model.train(True)  # Set model to training mode

    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
     
    for epoch in range(1, num_epochs+1):     
        correct = 0 
        total = 0
        epoch_loss = 0
        for data in s_dataloader:                                
            # get the inputs
            inputs, labels, weather = data
            now_batch_size,c,h,w = inputs.shape
            if now_batch_size<batchsize: # skip the last batch
                continue
            wt = label2tensor(weather)
            if torch.cuda.is_available():
                inputs = inputs.cuda().detach()
                labels = labels.cuda().detach()
                wt = wt.cuda().detach()

            # zero the parameter gradients
            optimizer.zero_grad()

            wo = model(inputs)
            loss = criterion(wo, wt)
            sssss = tensor2label(wt)
            xxxxx = tensor2label(wo)

            for i in range(batchsize):
                if sssss[i] == xxxxx[i]:
                    correct = correct + 1
            total = total + batchsize
            epoch_loss += float(loss.cpu().detach().numpy())
            loss.backward()
            optimizer.step()  
        print("Epoch: {} Loss: {:.4f} Acc: {:.4f}".format(epoch, epoch_loss, correct/total))

    # save pth
    if not os.path.isdir(os.path.join(os.getcwd(), "model", name)):
        os.mkdir(os.path.join(os.getcwd(), "model", name))
    save_filename = 'net_{:03d}_{}x{}.pth'.format(epoch, w, h)
    save_path = os.path.join(os.path.join(os.getcwd(), "model", name, save_filename))
    torch.save(model.cpu().state_dict(), save_path)

In [62]:
pretrain_WeatherCNN(train_num=32, w=256, h=256)

Epoch: 1 Loss: 4.6343 Acc: 0.1562
Epoch: 2 Loss: 4.6098 Acc: 0.1562
Epoch: 3 Loss: 4.9223 Acc: 0.0000
Epoch: 4 Loss: 4.7973 Acc: 0.0625
Epoch: 5 Loss: 4.7973 Acc: 0.0625
Epoch: 6 Loss: 4.7973 Acc: 0.0625
Epoch: 7 Loss: 4.4223 Acc: 0.2500
Epoch: 8 Loss: 4.4223 Acc: 0.2500
Epoch: 9 Loss: 4.6723 Acc: 0.1250
Epoch: 10 Loss: 4.8598 Acc: 0.0312
Epoch: 11 Loss: 4.7348 Acc: 0.0938
Epoch: 12 Loss: 4.8598 Acc: 0.0312
Epoch: 13 Loss: 4.7973 Acc: 0.0625
Epoch: 14 Loss: 4.7348 Acc: 0.0938
Epoch: 15 Loss: 4.6098 Acc: 0.1562
Epoch: 16 Loss: 4.7348 Acc: 0.0938
Epoch: 17 Loss: 4.6723 Acc: 0.1250
Epoch: 18 Loss: 4.8598 Acc: 0.0312
Epoch: 19 Loss: 4.9223 Acc: 0.0000
Epoch: 20 Loss: 4.6098 Acc: 0.1562
Epoch: 21 Loss: 4.6098 Acc: 0.1562
Epoch: 22 Loss: 4.8598 Acc: 0.0312
Epoch: 23 Loss: 4.6723 Acc: 0.1250
Epoch: 24 Loss: 4.6723 Acc: 0.1250
Epoch: 25 Loss: 4.8598 Acc: 0.0312
Epoch: 26 Loss: 4.7348 Acc: 0.0938
Epoch: 27 Loss: 4.6723 Acc: 0.1250
Epoch: 28 Loss: 4.6723 Acc: 0.1250
Epoch: 29 Loss: 4.7973 Acc: 0